In [1]:
# Import required modules
from langchain import hub
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain.llms import Ollama
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import streamlit as st
import os
import time

In [2]:
FILEPATH    = "resume6.pdf" 
LOCAL_MODEL = "llama3"
EMBEDDING   = "nomic-embed-text"

In [3]:
loader = PyPDFLoader(FILEPATH)
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)

In [4]:
persist_directory = 'data'

vectorstore = Chroma.from_documents(
    documents=all_splits, 
    embedding=OllamaEmbeddings(model=EMBEDDING),
    persist_directory=persist_directory
)

# vectorstore.persist()

C:\Users\Asus\AppData\Local\Temp\ipykernel_24584\2895862556.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model=EMBEDDING),


In [5]:
# llm = Ollama(base_url="http://localhost:11434",
#                                   model=LOCAL_MODEL,
#                                   verbose=True,
#                                   callback_manager=CallbackManager(
#                                       [StreamingStdOutCallbackHandler()])
#                                   )

# retriever = vectorstore.as_retriever()


llm = Ollama(
    base_url="http://localhost:11434",
    model=LOCAL_MODEL,
    verbose=True,
    callbacks=[StreamingStdOutCallbackHandler()]  # Use `callbacks` instead of `callback_manager`
)

retriever = vectorstore.as_retriever()

C:\Users\Asus\AppData\Local\Temp\ipykernel_24584\3370977395.py:11: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


In [6]:
template = """ You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative.
    
    Context: {context}
    History: {history}

    User: {question}
    Chatbot:
    """
prompt = PromptTemplate(
        input_variables=["history", "context", "question"],
        template=template,
    )

memory = ConversationBufferMemory(
        memory_key="history",
        return_messages=True,
        input_key="question"
    )


C:\Users\Asus\AppData\Local\Temp\ipykernel_24584\2668035149.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [7]:
qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type='stuff',
            retriever=retriever,
            verbose=True,
            chain_type_kwargs={
                "verbose": True,
                "prompt": prompt,
                "memory": memory,
            }
        )

In [8]:
query = "what is the GPA of harsh in MAster of computer Application"
query += ". Only from this pdf. Keep it short"
# response = qa_chain(query)

In [9]:
qa_chain.invoke({"query": query})



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative.
    
    Context: HARSH GUPTA 
+91-8825334559 | guptaharsh137@gmail.com| LinkedIn |GitHub 
 
EDUCATION 
 
Jain (Deemed to be University) (Present)  
Master of Computer Application – AI/ML                                                                                                                        Current 
CGPA 7.8 
Lovely Professional University  May 2023 
Bachelor of Computer Application – Application Development  GPA 8.55 
 
SKILLS 
 
Technical: Data Structures and Algorithms(intermediate) ,Object Oriented Programming(intermediate), Machine 
Learning 
Languages:  C++(intermediate) , Python(intermediate), Java(Beginner)  
Frontend:  ReactJs , NextJs ,HTML5, TailwindCSS  
Backend: NodeJs , Express ,  Mongooes 
Da

{'query': 'what is the GPA of harsh in MAster of computer Application. Only from this pdf. Keep it short',
 'result': "According to the provided PDF, Harsh Gupta's CGPA (Cumulative Grade Point Average) for Master of Computer Application – AI/ML is 7.8."}